# langChain Quickstart

## Initialization
### Environment and APIs

In [68]:
from dotenv import load_dotenv
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

load_dotenv()

# import os
# print ('LANGCHAIN_API_KEY: ', os.getenv('LANGCHAIN_API_KEY'))
# print ('OPENAI_API_KEY: ', os.getenv('OPENAI_API_KEY'))
# print ('TAVILY_API_KEY: ', os.getenv('TAVILY_API_KEY'))

True

### Models

In [69]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatOpenAI()
output_parser = StrOutputParser()


## Docstore pre-load

In [70]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()

docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


## Chains - Document Retrieval

In [71]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

doc_prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(chat_model, doc_prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

_TEST - retrieval_chain_

In [ ]:
retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

## Chains - History Aware Retrieval
### Generate search from history

In [72]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

search_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to lookup in order to get information relevant to the conversation.")
])
history_retriever_chain = create_history_aware_retriever(chat_model, retriever, search_prompt)


### Generate answer from history

In [73]:
history_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(chat_model, history_prompt)
history_chain = create_retrieval_chain(history_retriever_chain, document_chain)


_TEST - history_chain_

In [74]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fi

## Agent
### Search tool

In [75]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

tools = [retriever_tool, search]

In [103]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.utils.function_calling import format_tool_to_openai_tool
from langchain import hub
# from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

agent_prompt = hub.pull("hwchase17/openai-functions-agent")
agent_prompt.input_variables[1:1]=["chat_history"]
agent_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_chat_model_with_tools = agent_chat_model.bind(
        tools=[format_tool_to_openai_tool(tool) for tool in tools]
)
# agent = create_openai_functions_agent(agent_chat_model, tools, agent_prompt)
agent = (
    {
        "input": lambda x: x['input'],
        "agent_scratchpad": lambda x:
        format_to_openai_tool_messages(
                x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | agent_prompt
    | agent_chat_model_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [99]:
agent_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')

In [104]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...),
  chat_history: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messag

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

### langServe client test

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []  # Providing an empty list as this is the first call
})